# U2L5S2 - Validating regression models for prediction

In [1]:
# Make code toggle-able for easier review.
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<i>The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a></i>.''')

In [2]:
import numpy as np
import pandas as pd

import re

from matplotlib import pyplot as plt
import seaborn as sns

import math
from sklearn import linear_model
from scipy import stats

import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

from IPython.display import display

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Original model from L4 regression challenge
$ Property Crime = Population + Population ^2 + Murder + Robbery $

In [3]:
crime = pd.read_csv('/Users/guest/Dropbox/Education/Thinkful/Unit 2/L4 - Linear Regression/ny_crime_clean.csv')
crime['pop_log'] = crime['population_wins'].apply(lambda x: np.log(x))
crime['pop_sq_log'] = crime['population_sq'].apply(lambda x: np.log(x))
crime['prop_crime_log'] = crime['property_crime_wins'].apply(lambda x: np.log(x))

In [4]:
# Instantiate and fit our model
original_formula = 'prop_crime_log ~ pop_log+pop_sq_log+murder_cat+robbery_cat'
original_model = smf.ols(formula=original_formula, data=crime).fit()

In [5]:
# Inspect the model
original_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         prop_crime_log   R-squared:                       0.812
Model:                            OLS   Adj. R-squared:                  0.811
Method:                 Least Squares   F-statistic:                     496.4
Date:                Sat, 07 Jul 2018   Prob (F-statistic):          1.39e-124
Time:                        20:32:18   Log-Likelihood:                -333.96
No. Observations:                 348   AIC:                             675.9
Df Residuals:                     344   BIC:                             691.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -3.6410      0.347    -10.507      0.000      -4.323      -2.959
pop_log         0.1747      0.008     20.691      0.000       0.158       0.191
pop_sq_log      0.3494      0.017     20.691      0.000       0.316       0.383
murder_cat      0.2491      0.112      2.225      0.027       0.029       0.469
robbery_cat     0.8807      0.087     10.068      0.000       0.709       1.053
==============================================================================
Omnibus:                        6.569   Durbin-Watson:                   2.016
Prob(Omnibus):                  0.037   Jarque-Bera (JB):                6.348
Skew:                          -0.308   Prob(JB):                       0.0418
Kurtosis:                       3.240   Cond. No.                     1.03e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.35e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

On its face, murder appears to be less reliable than the other variables. It's significant (p  <0.05), but the confidence interval is pretty large. I'm going to cross-validate the model and see how it holds up.

In [6]:
# Define the cross_validation function
def cross_validation(dataframe, k, dv, ivs):

    frames = np.array_split(dataframe, k)     # Split frame into K folds  
        
    # Create dictionary to house cross validation results
    cv_dict = {'fold':[],
               'n':[],
               'R2':[]}
    for var in ivs: cv_dict[(var + '_p')] = []
        
    # Iterate through frames and record results
    label = 0     # Set fold label for easier reading

    for f in frames:
        
        # Create model from fold
        fold_formula = formula_from_list(dv,ivs)
        fold_model = smf.ols(formula=fold_formula, data=f).fit()

        # Record fold information
        label += 1 # Set fold label
        cv_dict['fold'].append(label) # Record fold label
        cv_dict['n'].append(f.shape[0]) # Record fold size
        
        # Record R squared for model
        cv_dict['R2'].append(round(float(fold_model.rsquared),3))

        # Record p values for independent variables
        pvalue_idx = 1 # set at one to skip intercept
        
        for var in fold_model.pvalues.index[1:]:
            cv_dict[(str(var) + '_p')].append(round(fold_model.pvalues[pvalue_idx],6))
            pvalue_idx += 1
        
    
    # Convert dictionary to dataframe
    summary_frame = pd.DataFrame(cv_dict, columns=list(cv_dict.keys())).set_index('fold')

    return summary_frame

In [7]:
# Define functions that allow for easy switching between formula format and list format
def list_from_formula(formula): return formula.split()[2].split('+')

def formula_from_list(dependent,independents):
    output_formula = dependent + ' ~ '
    for variable in independents:
        if variable == independents[0]:
            output_formula += variable
        else:
            output_formula += ('+' + variable)
    return output_formula

In [8]:
original_ivs = list_from_formula(original_formula)
cross_validation(crime,10,'prop_crime_log',original_ivs)

,n,R2,pop_log_p,pop_sq_log_p,murder_cat_p,robbery_cat_p
fold,,,,,,
1,35,0.909,0.000,0.000,0.152,0.001
2,35,0.810,0.000,0.000,0.672,0.004
3,35,0.673,0.000,0.000,0.625,0.000
4,35,0.867,0.000,0.000,0.715,0.017
5,35,0.831,0.000,0.000,0.342,0.000
6,35,0.801,0.000,0.000,0.035,0.001
7,35,0.798,0.000,0.000,0.518,0.015
8,35,0.863,0.000,0.000,0.339,0.011
9,34,0.770,0.000,0.000,0.886,0.003


R squared is reasonably consistent, but murder is not consistently significant across the folds. I'm going to try dropping it from my next iteration & checking both models against a new dataset.

## Second iteration of model
$ Property Crime = Population + Population ^2 + Robbery $

In [9]:
# Instantiate and fit our model
second_formula = 'prop_crime_log ~ pop_log+pop_sq_log+robbery_cat'
second_model = smf.ols(formula=second_formula, data=crime).fit()
second_ivs = list_from_formula(second_formula)

In [10]:
second_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         prop_crime_log   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.809
Method:                 Least Squares   F-statistic:                     733.7
Date:                Sat, 07 Jul 2018   Prob (F-statistic):          5.27e-125
Time:                        20:32:19   Log-Likelihood:                -336.44
No. Observations:                 348   AIC:                             678.9
Df Residuals:                     345   BIC:                             690.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -3.9367      0.322    -12.232      0.000      -4.570      -3.304
pop_log         0.1820      0.008     23.284      0.000       0.167       0.197
pop_sq_log      0.3640      0.016     23.284      0.000       0.333       0.395
robbery_cat     0.8862      0.088     10.077      0.000       0.713       1.059
==============================================================================
Omnibus:                        8.258   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.016   Jarque-Bera (JB):                8.135
Skew:                          -0.352   Prob(JB):                       0.0171
Kurtosis:                       3.258   Cond. No.                     9.47e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.58e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [11]:
cross_validation(crime,10,'prop_crime_log',second_ivs)

,n,R2,pop_log_p,pop_sq_log_p,robbery_cat_p
fold,,,,,
1,35,0.902,0.000,0.000,0.001
2,35,0.809,0.000,0.000,0.003
3,35,0.671,0.000,0.000,0.000
4,35,0.867,0.000,0.000,0.016
5,35,0.826,0.000,0.000,0.000
6,35,0.770,0.000,0.000,0.003
7,35,0.795,0.000,0.000,0.014
8,35,0.858,0.000,0.000,0.012
9,34,0.770,0.000,0.000,0.002


## Checking models against a different dataset

### Cleaning the new data

In [12]:
crime_2014_path = '/Users/guest/Dropbox/Education/Thinkful/Unit 2/L5 - Evaluating Linear Regression Models/Table_8_Offenses_Known_to_Law_Enforcement_by_New_York_by_City_2014.xls'
crime_2014_clean = '/Users/guest/Dropbox/Education/Thinkful/Unit 2/L5 - Evaluating Linear Regression Models/cleaned_2014_nyc_crime.csv'

In [13]:
crime2014 = pd.read_excel(crime_2014_path,
                          header=4,
                          skipfooter = 7)

In [14]:
# Tidy up the column names
crime2014.columns = crime2014.columns.str.strip().str.lower()

replacement_definitions = {' ':'_', 
                           '(':'',
                           ')':'',
                           '\n':'_',
                           '3':'',
                           '-':''}

for definition in replacement_definitions:
    crime2014.columns = crime2014.columns.str.replace(definition,replacement_definitions[definition])
    
# Set crime as the index
crime2014['city'] = crime2014['city'].apply(lambda x: re.sub('[^\D]',"",x))
crime2014 = crime2014.set_index('city')
    
# Replace nans with 0s
crime2014 = crime2014.fillna(value=0)

# Convert all values to floats
for col in crime2014.columns:
    try:
        crime2014[col] = crime2014[col].apply(lambda x: float(x.replace(',', '')))
    except:
        crime2014[col] = crime2014[col].apply(lambda x: float(x))
        
# Save the cleaned data as a CSV
crime2014.to_csv(crime_2014_clean)

In [15]:
list(crime2014)

['population',
 'violent_crime',
 'murder_and_nonnegligent_manslaughter',
 'rape_revised_definition1',
 'rape_legacy_definition2',
 'robbery',
 'aggravated_assault',
 'property_crime',
 'burglary',
 'larceny_theft',
 'motor_vehicle_theft',
 'arson']

In [16]:
# Create a function that will transform murder and robbery from continuous to categorical variables.

def cont_to_cat(x):
    if x > 0:
        return 1
    else:
        return 0

# Murder
crime2014['murder_cat'] = crime2014['murder_and_nonnegligent_manslaughter'].apply(cont_to_cat)

# Robbery
crime2014['robbery_cat'] = crime2014['robbery'].apply(cont_to_cat)

crime2014['population_wins'] = stats.mstats.winsorize(crime2014['population'], limits= 0.05)
crime2014['population_sq'] = crime2014['population_wins'] ** 2
crime2014['property_crime_wins'] = stats.mstats.winsorize(crime2014['property_crime'], limits= 0.05)
crime2014['pop_log'] = crime2014['population_wins'].apply(lambda x: np.log(x))
crime2014['pop_sq_log'] = crime2014['population_sq'].apply(lambda x: np.log(x))
crime2014['prop_crime_log'] = crime2014['property_crime_wins'].apply(lambda x: np.log(x))

###### Original model's performance on 2014 data

In [17]:
def performance_check(frame):
    for var in list(frame):
        print("{}'s standard deviation across folds: {}".format(var,round(frame[var].std(),4)))

In [18]:
original_cv = cross_validation(crime2014,10,'prop_crime_log',original_ivs)
original_cv

,n,R2,pop_log_p,pop_sq_log_p,murder_cat_p,robbery_cat_p
fold,,,,,,
1,37,0.903,0.000,0.000,0.438,0.002
2,37,0.830,0.000,0.000,0.883,0.031
3,37,0.491,0.003,0.003,0.210,0.114
4,37,0.831,0.000,0.000,0.963,0.095
5,37,0.734,0.001,0.001,0.112,0.002
6,37,0.760,0.005,0.005,0.096,0.000
7,37,0.807,0.001,0.001,0.116,0.003
8,37,0.755,0.000,0.000,0.773,0.359
9,37,0.761,0.001,0.001,0.374,0.166


In [19]:
performance_check(original_cv)

n's standard deviation across folds: 0.3162
R2's standard deviation across folds: 0.1142
pop_log_p's standard deviation across folds: 0.0017
pop_sq_log_p's standard deviation across folds: 0.0017
murder_cat_p's standard deviation across folds: 0.3437
robbery_cat_p's standard deviation across folds: 0.2687


###### Revised model's performance on 2014 data

In [20]:
new_cv = cross_validation(crime2014,10,'prop_crime_log',second_ivs)
new_cv

,n,R2,pop_log_p,pop_sq_log_p,robbery_cat_p
fold,,,,,
1,37,0.902,0.000,0.000,0.002
2,37,0.830,0.000,0.000,0.028
3,37,0.465,0.001,0.001,0.100
4,37,0.831,0.000,0.000,0.089
5,37,0.712,0.000,0.000,0.001
6,37,0.739,0.000,0.000,0.000
7,37,0.791,0.000,0.000,0.008
8,37,0.755,0.000,0.000,0.365
9,37,0.755,0.000,0.000,0.255


In [21]:
performance_check(new_cv)

n's standard deviation across folds: 0.3162
R2's standard deviation across folds: 0.1204
pop_log_p's standard deviation across folds: 0.0003
pop_sq_log_p's standard deviation across folds: 0.0003
robbery_cat_p's standard deviation across folds: 0.2714


## Interpretation

* Removing murder from the model seems to have been a good move – it's not significant across folds in 2014 in either model.
* R2's variance across folds is similar between the models – not perfect, but fairly consistent.
* Robbery is not significant across folds in 2014! This calls its value as a parameter into question. For a third iteration, I may consider dropping it and returning to the original dataset for more reliable potential parameters.